In [1]:
!pwd

/home/keti/cloud-edge-aicontainers/v2/bhc/webapp/EVC_RunTest


In [1]:
import yaml

with open('ServerConfig.yaml') as f:
    default_set = yaml.load(f, Loader=yaml.FullLoader)

default_cfg = default_set['autorun']

In [4]:
default_cfg

[{'name': 'activate AI model',
  'activation': 'run',
  'path': {'copy_playbook': '/home/keti/cloud-edge-aicontainers/v2/bhc/webapp/db/model/img_build/copy_model.yaml',
   'build_playbook': '/home/keti/cloud-edge-aicontainers/v2/bhc/webapp/db/model/img_build/autorun.yaml',
   'distrb_playbook': '/home/keti/cloud-edge-aicontainers/v2/bhc/webapp/db/model/run_model.yaml',
   'hosts_file': '/home/keti/cloud-edge-aicontainers/v2/bhc/webapp/hosts.ini',
   'db': '/home/keti/cloud-edge-aicontainers/v2/bhc/webapp/db/edge_logs.db3',
   'registry': '123.214.186.252:39500'}}]

In [9]:
default_cfg[0]['path']

{'copy_playbook': '/home/keti/cloud-edge-aicontainers/v2/bhc/webapp/db/model/img_build/copy_model.yaml',
 'build_playbook': '/home/keti/cloud-edge-aicontainers/v2/bhc/webapp/db/model/img_build/autorun.yaml',
 'distrb_playbook': '/home/keti/cloud-edge-aicontainers/v2/bhc/webapp/db/model/run_model.yaml',
 'hosts_file': '/home/keti/cloud-edge-aicontainers/v2/bhc/webapp/hosts.ini',
 'db': '/home/keti/cloud-edge-aicontainers/v2/bhc/webapp/db/edge_logs.db3',
 'registry': '123.214.186.252:39500'}

In [3]:
default_set['builders']

[{'name': 'rpi6401',
  'ip': '123.214.186.192',
  'port': 39001,
  'hw': 'rpi',
  'os': 'linux',
  'gpu': 'None'},
 {'name': 'n01',
  'ip': '123.214.186.192',
  'port': 33312,
  'hw': 'intel_NUC',
  'os': 'ubuntu',
  'gpu': 'RTX2080'}]

In [4]:
builders = []

for d in default_set['builders']:
    builders.append(d['name'])

print(builders)

['rpi6401', 'n01']


In [7]:
default_cfg[0]['activation']

'register'

In [3]:
import re

rows = []
p_start = re.compile('TASK \[get result].+')
p_end = re.compile('PLAY RECAP.+')

with open('modelinfo.txt', 'r') as f:

    lines = f.readlines()

## search task start line
    for line in lines:
        start = p_start.search(str(line))

        ## save index of string data
        if start:
            idx = lines.index(line)
            run = True

            while run:
                lines[idx] = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s\.\-\:\_]", "", lines[idx])
                lines[idx] = re.sub(" +", " ", lines[idx])
                lines[idx] = re.sub("\n", "", lines[idx])
                lines[idx] = re.sub("changed: ", "", lines[idx])
                lines[idx] = re.sub("ok: ", "", lines[idx])

                rows.append(lines[idx])

                end = p_end.search(lines[idx])

                idx += 1

                if end:
                    run = False

In [4]:
print(rows)

['TASK get result ', 'rpi6401 ', ' msg: ', ' date2023-06-17 03:20:57 0000 UTC', ' size2.14GB', ' repoaarch64-model', ' ', '', 'n01 ', ' msg: ', ' date2023-06-17 11:13:46 0900 KST', ' size8.68GB', ' repox86_64-model', ' ', '', '', 'PLAY RECAP ']


In [7]:
import os
import re

url = "https://github.com/ethicsense/evc-test3.git"
git_account = "ethicsense"

p_repo = re.compile("\/{git_account}\/.+\.git".format(git_account=git_account))
repo = p_repo.findall(url)[0]
repo = repo.replace(git_account, '')
repo = repo.replace('/', '')
repo = repo.replace('.git', '')

print(repo)

evc-test3


# 2023. 09. 06
# 모델 색인 절차 보완

In [4]:
import json
import os
import subprocess
import re

os.system('ansible-playbook ../playbooks/get_index.yaml -i ../hosts.ini -l rpi6401 > tmp/index.txt')

0

In [21]:
rows = []
p_start = re.compile('TASK \[get CPU architecture].+')
p_end = re.compile('PLAY RECAP.+')

with open('tmp/index.txt', 'r') as f:
    lines = f.readlines()

    for line in lines:
        start = p_start.search(str(line))

        if start:
            idx = lines.index(line)
            run = True
            idx += 1

            while run:
                lines[idx] = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s\.\-\:\_]", "", lines[idx])
                lines[idx] = re.sub(" +", " ", lines[idx])
                lines[idx] = re.sub("\n", "", lines[idx])
                lines[idx] = re.sub("changed.+", "", lines[idx])
                lines[idx] = re.sub("ok: ", "", lines[idx])
                lines[idx] = re.sub(" ansible_factsarchitecture: ", "", lines[idx])

                rows.append(lines[idx])
                end = p_end.search(lines[idx])
                idx += 1

                if end:
                    run = False
node = rows[0]
arch = rows[1]

print("target node :", node)
print("cpu architecture :", arch)


target node : rpi6401 
cpu architecture : aarch64


# 2023 10 31
## 그라디오 내부 기능 실험

In [1]:
node = "rpi6401"
ip = "12.3.1.5"
port = 3333
app = "12.3.1.5:3333"

node_list = []
n = [node, ip, port, app]
node_list.append(n)

In [2]:
print(node_list)

[['rpi6401', '12.3.1.5', 3333, '12.3.1.5:3333']]


In [3]:
txt = "helloworld"

helloworld


In [4]:
txt

'None'

# 2023 12 13
## 로그 전처리 및 DB 기록 코드부 수정

In [11]:
import re

rows = []
p_start = re.compile('TASK \[get result].+')
p_end = re.compile('PLAY RECAP.+')

with open('tmp/modelinfo.txt', 'r') as f:

            lines = f.readlines()

        ## search task start line
            for line in lines:
                start = p_start.search(str(line))

                ## save index of string data
                if start:
                    idx = lines.index(line)
                    run = True

                    while run:
                        lines[idx] = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s\.\-\:\_]", "", lines[idx])
                        lines[idx] = re.sub(" +", " ", lines[idx])
                        lines[idx] = re.sub("\n", "", lines[idx])
                        lines[idx] = re.sub("changed.+", "", lines[idx])
                        lines[idx] = re.sub("ok.+", "", lines[idx])

                        rows.append(lines[idx])

                        end = p_end.search(lines[idx])

                        idx += 1

                        if end:
                            run = False

In [12]:
rows

['TASK get result ',
 '',
 ' msg:',
 ' - date2023-12-13 11:24:44 0900 KST',
 ' - size9.85GB',
 ' - repox86_64-model',
 '',
 'PLAY RECAP ']

In [14]:
lst = []
p_date = re.compile('\d+-\d+-\d+ \d+:\d+:\d+')

for row in rows:
    if 'date' in row:
        tmp = p_date.findall(row)
        print(tmp)
        lst.append(tmp[0])

    elif 'size' in row:
        row = row.strip(' - size')
        row = row.strip('GB')
        lst.append(row)

    elif 'repo' in row:
        row = row.strip(' - repo')
        lst.append(row)

['2023-12-13 11:24:44']


In [15]:
lst

['2023-12-13 11:24:44', '9.85', 'x86_64-model']